In [1]:
%load_ext memory_profiler

# Dask DataFrames

<img src="dask-dataframe.svg" 
     align="right"
     width="40%"
     alt="Dask dataframes are blocked Pandas dataframes">
     
Dask Dataframes coordinate many Pandas dataframes, partitioned along an index.  They support a large subset of the Pandas API.

[References](https://examples.dask.org/dataframe.html)

In [2]:
from dask.distributed import Client, progress
#from dask_jobqueue import SLURMCluster
#cluster = SLURMCluster(cores=1,memory='6g')
#cluster.scale(2)
#client = Client(cluster)
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35573 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 33.73 GB


In [3]:
import dask.dataframe as dd

In [5]:
dask_df = dd.read_csv("OP_DTL_GNRL_PGYR2019_P06302020.csv",dtype='str')

In [6]:
dask_df.columns

Index(['Change_Type', 'Covered_Recipient_Type', 'Teaching_Hospital_CCN',
       'Teaching_Hospital_ID', 'Teaching_Hospital_Name',
       'Physician_Profile_ID', 'Physician_First_Name', 'Physician_Middle_Name',
       'Physician_Last_Name', 'Physician_Name_Suffix',
       'Recipient_Primary_Business_Street_Address_Line1',
       'Recipient_Primary_Business_Street_Address_Line2', 'Recipient_City',
       'Recipient_State', 'Recipient_Zip_Code', 'Recipient_Country',
       'Recipient_Province', 'Recipient_Postal_Code', 'Physician_Primary_Type',
       'Physician_Specialty', 'Physician_License_State_code1',
       'Physician_License_State_code2', 'Physician_License_State_code3',
       'Physician_License_State_code4', 'Physician_License_State_code5',
       'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
       'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name',
       'Applicable_Manufacturer_o

In [7]:
%%time
df_1 = dask_df['Total_Amount_of_Payment_USDollars'].astype(float)
df_1.describe().compute()

CPU times: user 6.61 s, sys: 405 ms, total: 7.01 s
Wall time: 1min 18s


count    1.036731e+07
mean     3.437341e+02
std      1.140449e+05
min      1.000000e-02
25%      1.433000e+01
50%      1.971000e+01
75%      1.235000e+02
max      3.000000e+08
Name: Total_Amount_of_Payment_USDollars, dtype: float64

In [8]:
dask_df["Total_Amount_of_Payment_USDollars"] = dask_df["Total_Amount_of_Payment_USDollars"].astype(float)

In [9]:
df_2 = dask_df.groupby(by=["Covered_Recipient_Type","Recipient_City"]).agg({"Total_Amount_of_Payment_USDollars": ['mean']})

In [10]:
%%time
df_2.compute()

CPU times: user 4.59 s, sys: 471 ms, total: 5.06 s
Wall time: 1min 8s


Total_Amount_of_Payment_USDollars
                                                                                     mean
Covered_Recipient_Type              Recipient_City                                       
Covered Recipient Physician         00612                                        8.251111
                                    100 HIGH ST                                141.304444
                                    11201 BENTON STREET                        250.000000
                                    150                                         22.480000
                                    1515 E Columbia                            499.500000
...                                                                                   ...
Covered Recipient Teaching Hospital chicago                                   3000.000000
                                    phoenix                                    500.000000
                                    sacramento                               14700.000000
                                    salt lake city                            1250.000000
                                    san diego                                 1000.000000

[20923 rows x 1 columns]

In [12]:
%%time
dask_df["Covered_Recipient_Type"].value_counts().compute()

CPU times: user 5.35 s, sys: 301 ms, total: 5.65 s
Wall time: 1min 7s


Covered Recipient Physician            10319667
Covered Recipient Teaching Hospital       47640
Name: Covered_Recipient_Type, dtype: int64